In [2]:
from multiprocessing import freeze_support
import pandas as pd
import string
import nltk
import re
import matplotlib.pyplot as plt
import gensim.corpora as corpora
import gensim
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
from gensim.models import Phrases, CoherenceModel
from nltk.tokenize import word_tokenize
from gensim.test.utils import datapath
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.cm as cm
from sklearn.manifold import TSNE
from gensim.corpora import Dictionary
import pickle
from gensim.test.utils import datapath

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
nltk.download('wordnet')
nltk.download('punkt')

wordnet_lemmatizer = WordNetLemmatizer()
wpt = nltk.WordPunctTokenizer()
lemmatizer = nltk.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zesha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
newStopWords = ['u','www','com','ca','xa0','b','age', 'color', 'national', 'origin', 'citizenship', 'physical', 'mental', 'disability', 'race', 'religion', 'creed', 'gender', 'sex', 'sexual', 'orientation', 'gender', 'identity', 'expression', 'genetic', 'marital','veteran']
stopwords.extend(newStopWords)

In [4]:
### PREPROCESSING

def remove_punc(corpus):
    punc_free = "".join([i for i in corpus if i not in string.punctuation])
    return punc_free

def tokenization(corpus):
    tokens = wpt.tokenize(corpus)
    return tokens

def remove_stopwords(corpus):
    output = [i for i in corpus if i not in stopwords]
    return output

def lemmization(corpus):
    lemm = [wordnet_lemmatizer.lemmatize(word) for word in corpus]
    return lemm

def cleanResume(corpus):
    corpus = re.sub('httpS+s*', ' ', corpus)  # remove URLs
    corpus = re.sub('RT|cc', ' ', corpus)  # remove RT and cc
    corpus = re.sub('#S+', '', corpus)  # remove hashtags
    corpus = re.sub('@S+', '  ', corpus)  # remove mentions
    corpus = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[]^_`{|}~•’"""), ' ', corpus)  # remove punctuations
    return corpus

def preprocessing(resumes):
    resumes['cleaned'] = resumes.apply(lambda x: cleanResume(x))
    print("Cleaned corpus")
    resumes['cleaned'] = resumes['cleaned'].apply(lambda x:remove_punc(x))
    print("Removed punctuation")
    resumes['cleaned'] = resumes['cleaned'].apply(lambda x:x.lower())
    print("Lowercase")
    resumes['cleaned'] = resumes['cleaned'].apply(lambda x:tokenization(x))
    print("Tokenized corpus")
    resumes['cleaned'] = resumes['cleaned'].apply(lambda x:remove_stopwords(x))
    print("Removed Stopwords")
    resumes['cleaned'] = resumes['cleaned'].apply(lambda x:lemmization(x))
    print("Lemmatized corpus")
    resumes.head()
    return resumes

In [5]:
def TokenizeLemm(corpus_list):

    #Tokenise the corpus
    tokenized_corp = [word_tokenize(i) for i in corpus_list]

    # Create Dictionary
    id2word = corpora.Dictionary(tokenized_corp)

    #Remove words that don't feature 20 times and those that feature in over 50% of documents
    id2word.filter_extremes(no_below=20, no_above=0.5, keep_n=80000)

    texts = tokenized_corp

    # Term Document Frequency
    corpus_final = [id2word.doc2bow(text) for text in texts]
    
    return corpus_final, id2word, texts